In [0]:
catalog = "main"
schema = "dbdemos_aibi_cme_marketing_campaign"

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")

In [0]:
spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {catalog}.{schema}.raw_data
""")

In [0]:
current_user = spark.sql("SELECT session_user() AS current_user").collect()[0].current_user

In [0]:
import zipfile

zip_path = f"/Workspace/Users/{current_user}/UUIC_Workshop/data.zip"
extract_dir = f"/Volumes/{catalog}/{schema}/raw_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

files = dbutils.fs.ls(f"{extract_dir}")
display(files)

In [0]:
import re

volume_path = f"/Volumes/{catalog}/{schema}/raw_data"
files = dbutils.fs.ls(volume_path)
json_files = [f.path for f in files if f.path.endswith('.json')]

for file_path in json_files:
    file_name = file_path.split('/')[-1]
    table_name = re.sub(r'\.json$', '', file_name)
    df = spark.read.json(file_path)
    df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{table_name}")